# Downloading Data from Yahoo Finance

Before we can do anything interesting, we need to get data. Downloading it ourselves can be a pain so let's automated it!

In [1]:
from pprint import pprint
import numpy as np
# Let's import the yahoo finance API
import yahoo_finance

def download_data(ticker, start_date, end_date):
    # Do a lookup on a ticker
    share = yahoo_finance.Share(ticker)
    # And query the historical data
    return share.get_historical(start_date, end_date)

# And a little test with Walmart
pprint(download_data('WMT', '2016-02-10', '2016-02-16')[0])

{'Adj_Close': '65.415729',
 'Close': '65.900002',
 'Date': '2016-02-16',
 'High': '66.800003',
 'Low': '65.50',
 'Open': '66.610001',
 'Symbol': 'WMT',
 'Volume': '11360500'}


# Dealing with Multiple Stocks

The previous code will only allow us to query on a single stock, so let's wrap it so we can iterate over multiple stocks.

In [2]:
def download_from_yf(tickers, start_date, end_date):
    data = {}
    for ticker in tickers:
        data[ticker] = download_data(ticker, start_date, end_date)
    return data

In [3]:
# We need to tell the downloader what stocks we care about!
tickers = ['WMT', 'XOM'] # Walmart and ExxonMobil from example in class

# And also a timerange, i.e. the last month
import datetime
import dateutil.relativedelta

end_date = datetime.date.today()

# And now we just have to subtract a month from end_date
start_date = end_date - dateutil.relativedelta.relativedelta(months=1)
print "Fetch stock data for %s from"%", ".join(tickers), start_date, "to", end_date



Fetch stock data for WMT, XOM from 2016-02-23 to 2016-03-23


But the Yahoo Finance API wants the dates as strings, so we need to convert from a datetime.date object to a string!
* The string is formatted as YYYY-MM-DD (which also happens to be the default format Python uses)

In [4]:
date_format = '%Y-%m-%d' # YYYY-MM-DD in Python
s_date = start_date.strftime(date_format)

# We could have alternatively cast the dates into a
# string since %Y-%m-%d is the the default date format
# s_date = str(start_date)

e_date = end_date.strftime(date_format)

In [5]:
data = download_from_yf(tickers, s_date, e_date)
from pprint import pprint
pprint(data)

{'WMT': [{'Adj_Close': '67.870003',
          'Close': '67.870003',
          'Date': '2016-03-22',
          'High': '68.68',
          'Low': '67.790001',
          'Open': '68.00',
          'Symbol': 'WMT',
          'Volume': '7497600'},
         {'Adj_Close': '67.970001',
          'Close': '67.970001',
          'Date': '2016-03-21',
          'High': '68.379997',
          'Low': '66.970001',
          'Open': '67.029999',
          'Symbol': 'WMT',
          'Volume': '8978500'},
         {'Adj_Close': '66.949997',
          'Close': '66.949997',
          'Date': '2016-03-18',
          'High': '68.029999',
          'Low': '66.75',
          'Open': '67.480003',
          'Symbol': 'WMT',
          'Volume': '22837600'},
         {'Adj_Close': '67.449997',
          'Close': '67.449997',
          'Date': '2016-03-17',
          'High': '68.919998',
          'Low': '67.260002',
          'Open': '67.82',
          'Symbol': 'WMT',
          'Volume': '11823000'},
         {

# What if we want weekly data?

Unfortunately, YF doesn't seem to provide weekly and monthly data through their API. 

We need to generate this data ourselves

* Open Price is the opening price we observe on the Monday
* Similarly, closing price is determined by the closing price on Friday
 * Actually this isn't completely true due to fixed intervals, holidays, etc... but we will leave those details for the implementation
* Adjusted Close works the same as close
* Low is the lowest 'low' in a week
* High is the highest 'high' in the week
* Volume is average volume over the week
 * $Volume = \frac{1}{N} \sum_{i=1}^{N} Volume_i$
 * Yahoo Finance actually truncates the number since they track volume to the hundreds place

In [6]:
def convert_to_weekly(data):
    def __convert_to_weekly(data):
        # Sort the data by time from oldest date to most recent
        data = sorted(data, key=lambda row: datetime.datetime.strptime(row['Date'], date_format))
        weeks = {}
        
        # High level approach: Markets typically open on a Monday,
        # so we see if the Monday of a given week is in the weeks dict
        #   if it is: we aren't the first day the markets were open on a given week
        #     - Update adj. close and close
        #     - Append volume to list
        #     - And check low and high
        #   if it isn't: we are the first day
        #     - Do the above stuff and also set the open price
        
        for row in data:
            d = datetime.datetime.strptime(row['Date'], date_format)
            # Expected start of the week
            d_start = (d-datetime.timedelta(d.weekday())).strftime(date_format)
            if d_start not in weeks:
                # First day in a given week
                weeks[d_start] = row
                weeks[d_start]['Volume'] = [row['Volume']]
            else:
                # Append the daily volume to the weekly count
                weeks[d_start]['Volume'].append(row['Volume'])
                # Update close and adj. close since we are the latest date
                weeks[d_start]['Close'] = row['Close']
                weeks[d_start]['Adj_Close'] = row['Adj_Close']

                # Check if we are higher than the highest observed value
                if float(weeks[d_start]['High']) < float(row['High']):
                    weeks[d_start]['High'] = row['High']

                # Check if we are lower than the lowest observed value
                if float(weeks[d_start]['Low']) > float(row['Low']):
                    weeks[d_start]['Low'] = row['Low']

        rows = []
        for k,v in weeks.items():
            # Sum up the volumes and divide by the count (compute average volume)
            v['Volume'] = np.average(map(int, v['Volume']))
            rows.append(v)
        return rows
        
    for ticker in data.keys():
        data[ticker] = __convert_to_weekly(data[ticker])
    return data

In [7]:
weekly_data = convert_to_weekly(data)
pprint(weekly_data)


{'WMT': [{'Adj_Close': '66.021247',
          'Close': '66.510002',
          'Date': '2016-02-23',
          'High': '68.199997',
          'Low': '65.279999',
          'Open': '65.839996',
          'Symbol': 'WMT',
          'Volume': 10572200.0},
         {'Adj_Close': '66.289259',
          'Close': '66.779999',
          'Date': '2016-02-29',
          'High': '67.279999',
          'Low': '64.879997',
          'Open': '66.629997',
          'Symbol': 'WMT',
          'Volume': 10159520.0},
         {'Adj_Close': '67.870003',
          'Close': '67.870003',
          'Date': '2016-03-21',
          'High': '68.68',
          'Low': '66.970001',
          'Open': '67.029999',
          'Symbol': 'WMT',
          'Volume': 8238050.0},
         {'Adj_Close': '66.949997',
          'Close': '66.949997',
          'Date': '2016-03-14',
          'High': '68.919998',
          'Low': '66.75',
          'Open': '67.150002',
          'Symbol': 'WMT',
          'Volume': 11414840.0},
 

# Getting monthly data

It's easier than weekly since you don't have to deal with weekdays! I am not going to go into the details...

# Aside on Numerical Analysis & Processing

Python defaults to representing numbers as floating point numbers (floats). This is fine for many applications, but finance is not one of them. Many finance companies choose to work with integers (i.e. MasterCard) and convert 123.45 into 12345.

Floats are great because they can store data in a massive range: $1.175494 × 10^{-38} \leq x \leq 3.402823 × 10^{38}$, but they cannot represent every rational number or even every integer in that range. The easy way to think about it is there are a finite number of representations, which is much smaller than the range. To represent a number, we want to minimize the error between the actual number and it's representation. 

If we start doing lots of math with floating point number, we will start increasing the amount of error!


Look at some of the decimals above. Unfortunately, the Yahoo Finance API uses floats.

In [8]:
# Python's decimal class let's us represent decimals 
# with a fixed number of digits after the decimal place
import decimal
# This is no good as 73.9... is a float and we will represent it to 26 places of accuracy
dec = decimal.Decimal(73.910002)
print dec

# If we round it first, we get nicer results :)
dec = decimal.Decimal(str(round(73.910002, 2)))
print dec

73.9100020000000057507349993102252483367919921875
73.91


# Getting the data into Excel

We might want to leverage excel in our analysis. Fortunately Python has a module for working with excel!

In [9]:
# Let's import it and get started
import xlsxwriter

In [10]:
def write_xlsx(data, output_file):
    header = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj_Close']
    # Create a new workbook
    workbook = xlsxwriter.Workbook(output_file)
    # Pick the format that you want Excel to use for dates
    # mm/dd/yy is the default, so let's use that
    excel_date_format = workbook.add_format({'num_format': 'mm/dd/yy'})
    
    def __write_val(sheet, field, row, row_pos, col_pos):
        # A little helper func for dealing with different data types
        if field == 'Date':
            # Write the date to cell (col_pos, row_pos) i.e. (A1)
            sheet.write_datetime(
                row_pos,
                col_pos,
                datetime.datetime.strptime(row['Date'], date_format),
                excel_date_format
            )
        elif field == 'Volume':
            # Volume is an int, so we will write it as such
            sheet.write_number(row_pos, col_pos, int(row[field]))
        else:
            sheet.write_number(
                row_pos,
                col_pos,
                # From the aside
                decimal.Decimal(str(round(float(row[field]), 2)))
            )
    # Add the worksheets in sorted order
    for ticker in sorted(data.keys()):
        # Just name the worksheet after the stock
        worksheet = workbook.add_worksheet(ticker)
        row_pos = 0
        col_pos = 0
        # Write the header to the sheet
        for field in header:
            worksheet.write(row_pos, col_pos, field)
            col_pos += 1
        row_pos += 1
        
        # Sort the dates in descending order
        for row in sorted(data[ticker],
            key=lambda r: datetime.datetime.strptime(r['Date'], date_format)):
                col_pos = 0
                for field in header:
                    __write_val(worksheet, field, row, row_pos, col_pos)
                    col_pos += 1
                row_pos += 1

    # And finally close the workbook
    workbook.close()
        
    

In [11]:
# And let's try it
out_file = 'ticker_prices.xlsx'
write_xlsx(weekly_data, out_file)
# Should now have ticker_prices.xlsx in your working directory
import os
# Just a little check to see if the file is there
assert(out_file in os.listdir('.'))

# Conclusion

Open ticker_prices.xlsx up in excel and see the results. Feel free to tweak this notebook and experiment with it!

If we wanted to build a portfolio testing (backtesting) framework, we don't want to have to keep querying for data.
* Solution: Build a transparent wrapper around our queries to fetch them from a local cache if they exist, otherwise reach out to Yahoo Finance
 * A bit of overhead when populating the database, but much faster going forward